Factorization Machines
- a generic algorithm for classification, regression and ranking
- a generalization of linear regression and matrix factorization model
- similar to SVM with a polynomial kernel
- models n-way (typically 2-way) interaction between features (embeddings)
- compute 2-way feature interaction as the dot product of the latent features
- author proposes a neat trick to reduce polynomial complexity to linear time

the model (2-way interaction): 

$\hat{y}(x) = \mathbf{w}_0 + \sum_{i=1}^d \mathbf{w}_i x_i + \sum_{i=1}^d\sum_{j=i+1}^d \langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j$

- $w_0$ - global bias
- $w_i$ - weight of i-th variable
- $V, v_i, v_j$ - feature embeddings
- $\langle\mathbf{v}_i, \mathbf{v}_j\rangle$ - dot product of embeddings -> weight of the interaction

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import roc_auc_score

In [4]:
class FM(torch.nn.Module):
    def __init__(self,):
        super().__init__()
        